# Init

In [ ]:
from data.datasource.tv_data_feed_api import get_data
from tvDatafeed import TvDatafeed, Interval
import requests

import os
from datetime import datetime
from openpyxl import Workbook, load_workbook
import time
import pandas as pd
import matplotlib.pyplot as plt

import business.utils.trading_signals as ts
import business.utils.trading_indicators as ti

LOG_FILE = r"F:\Desktop\git\crypto_bi\log.xlsx"


# historical_data = get_data(symbol="BTCUSDT", exchange="BINANCE", interval=Interval.in_1_hour, n_bars=100000)
# historical_data

In [5]:
def send_notification(message):
    # Replace these with your actual Pushover API token and user key
    app_token = 'acgrip2rbsfh6cdnghafcc922btrht'  # e.g., "a1b2c3d4e5f6g7h8i9j0"
    user_key = 'ucj8xoq8a275k5g7cn6nh91sdfk53m'    # e.g., "u1v2w3x4y5z6"

    # Prepare the payload with a loud sound ('siren' is one example)
    payload = {
        'token': app_token,
        'user': user_key,
        'message': message,
        'sound': 'siren',  # Change this to any sound of your choice
        'priority': 1      # priority 1 or 2 can trigger an alert mode on some devices
    }
    
    # Send the POST request to Pushover
    response = requests.post("https://api.pushover.net/1/messages.json", data=payload)
    
    if response.status_code == 200:
        print("Notification sent successfully!")
    else:
        print("Failed to send notification. Response:", response.text)


In [6]:
def log_to_excel(timestamp, symbol, status, message, log_file=LOG_FILE):
    # If the log file doesn't exist, create it with headers
    if not os.path.exists(log_file):
        wb = Workbook()
        ws = wb.active
        ws.title = "Log"
        # Write header row
        ws.append(["Timestamp", "Symbol", "Status", "Message"])
        wb.save(log_file)
    
    # Load the existing workbook and append a new row
    wb = load_workbook(log_file)
    ws = wb.active
    ws.append([timestamp, symbol, status, message])
    wb.save(log_file)


# Live Signal Multiple Stocks

In [4]:
# List of symbols to monitor
symbols = ["BTCUSDT", "ETHUSDT"]

check_signals = [ts.very_low_price,
                    #very_high_price
                    ]

In [5]:
def run_checks_and_notify(price_threshold):
    for symbol in symbols:
        # Retrieve data for the symbol (adjust parameters as needed)
        df = get_data(
            symbol=symbol,
            exchange="BINANCE",
            interval=Interval.in_1_hour,
            n_bars=100
        )
        
        # Run each check function on the data
        for check_func in check_signals:
            status, message = check_func(symbol, df, price_threshold)
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            log_to_excel(current_time, symbol, status, message)
            if status:
                send_notification(message)


In [6]:
run_checks_and_notify(price_threshold=2560)